In [ ]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
ds_train, ds_test = tfds.load('ag_news_subset').values()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteUQT2GJ/ag_news_subset-train.tfrecord*...:  …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteUQT2GJ/ag_news_subset-test.tfrecord*...:   …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


##CBoW Model

In [ ]:
vocab_size = 5000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size, input_shape=(1,))
embedder = keras.layers.Embedding(vocab_size,30, input_length=1)

model = keras.Sequential([
    embedder,
    keras.layers.Dense(vocab_size, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 30)             150000    
                                                                 
 dense (Dense)               (None, 1, 5000)           155000    
                                                                 
Total params: 305000 (1.16 MB)
Trainable params: 305000 (1.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def extract_text(x):
    return x['title']+' '+x['description']

vectorizer.adapt(ds_train.take(500).map(extract_text))
vocab = vectorizer.get_vocabulary()

##Preparing Training Data

In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0, i-window_size), min(i+window_size+1, len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I', 'like','to','train','networks']))
print(to_cbow(vectorizer('I like to train networks')))


[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[<tf.Tensor: shape=(), dtype=int64, numpy=232>, <tf.Tensor: shape=(), dtype=int64, numpy=112>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=112>], [<tf.Tensor: shape=(), dtype=int64, numpy=112>, <tf.Tensor: shape=(), dtype=int64, numpy=232>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=232>], [<tf.Tensor: shape=(), dtype=int64, numpy=1388>, <tf.Tensor: shape=(), dtype=int64, numpy=232>], [<tf.Tensor: shape=(), dtype=int64, numpy=112>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=232>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=1388>, <tf.Tensor: shape=(), dtype=int64, num

In [ ]:
X =[]
Y =[]
for i,x in zip(range(10000), ds_train.map(extract_text).as_numpy_iterator()):
    for w1,w2 in to_cbow(vectorizer(x), window_size=1):
        X.append(tf.expand_dims(w1,0))
        Y.append(tf.expand_dims(w2,0))

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((X,Y)).batch(256)

In [ ]:
model.compile(optimizers=keras.optimizers.SGD(lr=0.1),loss='sparse_categorical_crossentropy')
model.fit(ds, epochs=200)

TypeError: Invalid keyword argument(s) in `compile()`: ({'optimizers'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

##Trying out Word2vec

In [ ]:
vectors = embedder(vectorizer(vocab))
vectors = tf.reshape(vectors,(-1,30))

In [ ]:
paris_vec = embedder(vectorizer('paris'))[0]
print(paris_vec)

In [ ]:
def close_words(x,n=5):
    vec = embedder(vectorizer(x))[0]
    top5 = np.linalg.norm(vectors-vec, axis=1).argsort()[:n]
    return [vocab[x] for x in top5]

close_words('paris')

In [ ]:
close_words('china')

In [ ]:
close_words('official')